## Code for [GEO-AI Challenge for Cropland Mapping by ITU](https://zindi.africa/competitions/geo-ai-challenge-for-cropland-mapping-with-satellite-imagery) on ZINDI
#### Author: Maryam Salehi, PhD in Remote Sensing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/zindi/crop

/content/drive/MyDrive/Colab Notebooks/zindi/crop


### <font color="brown"> Import packages

In [ ]:
#pip install geemap

In [ ]:
import ee
import numpy as np
import pandas as pd
import random

import geemap

In [ ]:
# Get authetication token and sign in to Google Earth Engine
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=PIa-Mz9_wFCdAJVgbe8un6VOh3EvUz1gTtyX2JbGUTc&tc=l-JH1xsnJXKpvI31h_35-cMPMj2mevDA-wPfn7F9giA&cc=_Td312gE9Pk4TL7NG1l_Ij6ndHNobJAdouesvPHWgL4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlPbd9hGUqS9ulYeRxAmUMWbXm8Wk64fXg3e9-Lv8K1WaFxlMGvYfE

Successfully saved authorization token.


In [ ]:
# Set seed for reproducability
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

### <font color="brown"> Load Data

In [ ]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# Preview head of train
train.head()

,ID,Lat,Lon,Target
0,ID_SJ098E7S2SY9,34.162491,70.763668,0
1,ID_CWCD60FGJJYY,32.075695,48.492047,0
2,ID_R1XF70RMVGL3,14.542826,33.313483,1
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1
4,ID_C20R2C0AYIT0,14.419128,33.528450,0


### <font color="brown"> Visualize the data 👀

In [ ]:
# Remove cloud and shadow pixels from Sentinel-2 SR image
def maskCloudAndShadowsSR(image):
    cloudProb = image.select('MSK_CLDPRB');
    cloud = cloudProb.lt(10);
    scl = image.select('SCL');
    shadow = scl.eq(3);  # 3 = cloud shadow
    cirrus = scl.eq(10); # 10 = cirrus
    mask = cloud.And(cirrus.neq(1)).And(shadow.neq(1));
    return image.updateMask(mask)

In [ ]:
# Visualize Sentinel-2 data

iran_bbox = ee.Geometry.Rectangle([44.036979, 25.0695, 63.31663, 39.71393])
sudan_bbox = ee.Geometry.Rectangle([21.820312, 8.6753983, 38.420898, 22.232395])
afg_bbox = ee.Geometry.Rectangle([60.4713, 29.377, 75.1580, 38.486])

image_afghanistan = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2022-04-01', '2022-04-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskCloudAndShadowsSR).select('B.*')
image_iran = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2019-07-01', '2020-06-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskCloudAndShadowsSR).select('B.*')
image_sudan = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2019-07-01', '2020-06-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskCloudAndShadowsSR).select('B.*')

# Get the image's band names
band_names = image_afghanistan.first().bandNames().getInfo()
print('Band names: ', band_names)

visualization = {'min': 0,'max': 3000,'bands': ['B4', 'B3', 'B2']}

Map = geemap.Map()
Map.setCenter(48, 32, 5)
Map.addLayer(image_iran.mean().clip(iran_bbox), visualization, 'RGB_iran')
Map.addLayer(image_sudan.mean().clip(sudan_bbox), visualization, 'RGB_sudan')
Map.addLayer(image_afghanistan.mean().clip(afg_bbox), visualization, 'RGB_afghanistan')
Map

Band names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


Map(center=[32, 48], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [ ]:
# Create a list to store the points
points = []

# Iterate over each row in train data
for index, row in train.iterrows():
    # Get the latitude and longitude values
    lat = row['Lat']
    lon = row['Lon']

    # Create a point geometry
    point = ee.Geometry.Point(lon, lat)

    # Add the point to the list
    points.append(point)

# Convert the list of points to an Earth Engine FeatureCollection
fc = ee.FeatureCollection(points)
# Add the points and visualization to the map
Map.addLayer(fc, {}, 'Points')
Map

Map(bottom=3627.0, center=[32, 48], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [ ]:
# Visualize Sentinel-1 data

s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2022-04-01', '2022-04-30')

s1Dual = (s1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) # Filter to get images with VV and VH dual polarization
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
           .filter(ee.Filter.eq('instrumentMode', 'IW')))

# Filter to get images from different look angles
s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# Create a composite from means at different polarizations and look angles
composite = ee.Image.cat([
  s1DualAscending.select('VH').mean(),
  ee.ImageCollection(s1DualAscending.select('VV').merge(s1DualDescending.select('VV'))).mean(),
  s1DualDescending.select('VH').mean()
]).focal_median()

# Display as a composite of polarization and backscattering characteristics
Map = geemap.Map()
Map.setCenter(48, 32, 5)
Map.addLayer(composite, {'min': [-25, -20, -25], 'max': [0, 10, 0]}, 'composite')
Map

Map(center=[32, 48], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

# <font color="orange"> Sentinel data preprocessing

In [ ]:
data = train.copy()

# Define the latitude and longitude ranges for each country
afghanistan_lat_range = (30, 40)
afghanistan_lon_range = (65, 75)

iran_lat_range = (25, 40)
iran_lon_range = (40, 65)

sudan_lat_range = (10, 25)
sudan_lon_range = (20, 40)

# Filter the data for Afghanistan
afghanistan_data = data[(data['Lat'] >= afghanistan_lat_range[0]) & (data['Lat'] <= afghanistan_lat_range[1]) &
                        (data['Lon'] >= afghanistan_lon_range[0]) & (data['Lon'] <= afghanistan_lon_range[1])]

# Filter the data for Iran
iran_data = data[(data['Lat'] >= iran_lat_range[0]) & (data['Lat'] <= iran_lat_range[1]) &
                 (data['Lon'] >= iran_lon_range[0]) & (data['Lon'] <= iran_lon_range[1])]

# Filter the data for Sudan
sudan_data = data[(data['Lat'] >= sudan_lat_range[0]) & (data['Lat'] <= sudan_lat_range[1]) &
                  (data['Lon'] >= sudan_lon_range[0]) & (data['Lon'] <= sudan_lon_range[1])]

In [ ]:
# Define a function to calculate a single average value for the entire time (Sentinel-2)

s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

bands = ['B1', 'B2', 'B3', 'B4','B5', 'B6', 'B7', 'B8','B8A', 'B9', 'B11', 'B12']

# Define the date range for each country
start_date_afghanistan = ee.Date('2022-04-01')
end_date_afghanistan = ee.Date('2022-04-30')

start_date_iran = ee.Date('2019-07-01')
end_date_iran = ee.Date('2020-06-30')

start_date_sudan = ee.Date('2019-07-01')
end_date_sudan = ee.Date('2020-06-30')

# Function to extract mean pixel values for a labeled location
def extract_mean_pixel_values_afghanistan(row):
    lon = row['Lon']
    lat = row['Lat']

    # Create a point geometry for the labeled location
    point = ee.Geometry.Point(lon, lat)

    # Filter the S2 collection by the location and desired time range
    s2_filtered = s2_collection.filterBounds(point).filterDate(start_date_afghanistan, end_date_afghanistan).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskCloudAndShadowsSR).select('B.*')

    # Calculate the mean pixel values for the bands of interest at the labeled location
    mean_values = s2_filtered.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10
    )

    # Extract the mean pixel values for the bands
    values = [mean_values.get(band).getInfo() for band in bands]

    return values

def extract_mean_pixel_values_iran(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    s2_filtered = s2_collection.filterBounds(point).filterDate(start_date_iran, end_date_iran).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskCloudAndShadowsSR).select('B.*')

    mean_values = s2_filtered.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10
    )

    values = [mean_values.get(band).getInfo() for band in bands]

    return values

def extract_mean_pixel_values_sudan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    s2_filtered = s2_collection.filterBounds(point).filterDate(start_date_sudan, end_date_sudan).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskCloudAndShadowsSR).select('B.*')

    mean_values = s2_filtered.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10
    )

    values = [mean_values.get(band).getInfo() for band in bands]

    return values

In [ ]:
# Define a function to calculate mean values for each month of a year (Sentinel-2)

s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
bands = ['B1', 'B2', 'B3', 'B4','B5', 'B6', 'B7', 'B8','B8A', 'B9', 'B11', 'B12']

start_date_afghanistan = ee.Date('2021-04-01')
end_date_afghanistan = ee.Date('2022-03-31')

def extract_monthly_mean_pixel_values_afghanistan(row):
    lon = row['Lon']
    lat = row['Lat']

    # Create a point geometry for the labeled location
    point = ee.Geometry.Point(lon, lat)

    # Initialize an empty list to store monthly mean values
    monthly_mean_values = []

    # Loop through each month in the date range
    current_date = start_date_afghanistan

    while current_date.getInfo()['value'] <= end_date_afghanistan.getInfo()['value']:

        # Filter the S2 collection by the location and the current month
        s2_filtered = s2_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskCloudAndShadowsSR).select('B.*')

        # Calculate the mean pixel values for the current month
        mean_values = s2_filtered.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=10
        )

        if s2_filtered.size().getInfo() == 0:
           mean_values = None

        if mean_values is not None:
          values = [mean_values.get(band).getInfo() for band in bands]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')

    return monthly_mean_values

def extract_monthly_mean_pixel_values_iran(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_iran

    while current_date.getInfo()['value'] <= end_date_iran.getInfo()['value']:

        s2_filtered = s2_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskCloudAndShadowsSR).select('B.*')

        mean_values = s2_filtered.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=10
        )

        values = [mean_values.get(band).getInfo() for band in bands]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')

    return monthly_mean_values

def extract_monthly_mean_pixel_values_sudan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_sudan

    while current_date.getInfo()['value'] <= end_date_sudan.getInfo()['value']:

        s2_filtered = s2_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskCloudAndShadowsSR).select('B.*')

        mean_values = s2_filtered.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=10
        )

        values = [mean_values.get(band).getInfo() for band in bands]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')

    return monthly_mean_values

In [ ]:
# Define a function to calculate a single average value for the entire time (Sentinel-1)

s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD')

#***********************************************************************#
bands = ['vhAsc', 'vhDesc', 'VVAscDesc']   # for iran and afghanistan
#bands = ['vhDesc', 'VVAscDesc']           # for sudan
#***********************************************************************#

start_date_afghanistan = ee.Date('2022-04-01')
end_date_afghanistan = ee.Date('2022-04-30')

start_date_iran = ee.Date('2019-07-01')
end_date_iran = ee.Date('2020-06-30')

start_date_sudan = ee.Date('2019-07-01')
end_date_sudan = ee.Date('2020-06-30')

def extract_mean_pixel_values_afghanistan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    s1_filtered = s1_collection.filterBounds(point).filterDate(start_date_afghanistan, end_date_afghanistan)
    s1Dual = (s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) # Filter to get images with VV and VH dual polarization
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')))
    # Filter to get images from different look angles
    s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

    # Extract the mean pixel values for the bands
    vhIwAscMean = s1DualAscending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
    vhIwDescMean = s1DualDescending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
    vvIwAscDescMean = s1DualAscending.merge(s1DualDescending).select('VV').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VV').getInfo()
    values = [vhIwAscMean,vhIwDescMean,vvIwAscDescMean]

    return values

def extract_mean_pixel_values_iran(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    s1_filtered = s1_collection.filterBounds(point).filterDate(start_date_iran, end_date_iran)
    s1Dual = (s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')))
    s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    vhIwAscMean = s1DualAscending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
    vhIwDescMean = s1DualDescending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
    vvIwAscDescMean = s1DualAscending.merge(s1DualDescending).select('VV').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VV').getInfo()
    values = [vhIwAscMean,vhIwDescMean,vvIwAscDescMean]

    return values

def extract_mean_pixel_values_sudan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    s1_filtered = s1_collection.filterBounds(point).filterDate(start_date_sudan, end_date_sudan)
    s1Dual = (s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')))
    s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    #vhIwAscMean = s1DualAscending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()  # no data
    vhIwDescMean = s1DualDescending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
    vvIwAscDescMean = s1DualAscending.merge(s1DualDescending).select('VV').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VV').getInfo()
    values = [vhIwDescMean,vvIwAscDescMean]
    #values = [vhIwAscMean,vhIwDescMean,vvIwAscDescMean]

    return values

In [ ]:
# Define a function to calculate mean values for each month of a year (Sentinel-1)

s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD')

#*********************************************************************#
bands = ['vhAsc', 'vhDesc', 'VVAscDesc']   # for iran and afghanistan
#bands = ['vhDesc', 'VVAscDesc']           # for sudan
#*********************************************************************#

start_date_afghanistan = ee.Date('2021-04-01')
end_date_afghanistan = ee.Date('2022-03-31')

def extract_monthly_mean_pixel_values_afghanistan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_afghanistan

    while current_date.getInfo()['value'] <= end_date_afghanistan.getInfo()['value']:

        s1_filtered = s1_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month'))
        s1Dual = (s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')))
        s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
        s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
        vhIwAscMean = s1DualAscending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
        vhIwDescMean = s1DualDescending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
        vvIwAscDescMean = s1DualAscending.merge(s1DualDescending).select('VV').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VV').getInfo()
        values = [vhIwAscMean,vhIwDescMean,vvIwAscDescMean]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')

    return monthly_mean_values

def extract_monthly_mean_pixel_values_iran(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_iran

    while current_date.getInfo()['value'] <= end_date_iran.getInfo()['value']:

        s1_filtered = s1_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month'))
        s1Dual = (s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')))
        s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
        s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
        vhIwAscMean = s1DualAscending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
        vhIwDescMean = s1DualDescending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
        vvIwAscDescMean = s1DualAscending.merge(s1DualDescending).select('VV').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VV').getInfo()
        values = [vhIwAscMean,vhIwDescMean,vvIwAscDescMean]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')

    return monthly_mean_values

def extract_monthly_mean_pixel_values_sudan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_sudan

    while current_date.getInfo()['value'] <= end_date_sudan.getInfo()['value']:

        s1_filtered = s1_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month'))
        s1Dual = (s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')))
        s1DualAscending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
        s1DualDescending = s1Dual.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
        #vhIwAscMean = s1DualAscending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
        vhIwDescMean = s1DualDescending.select('VH').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VH').getInfo()
        vvIwAscDescMean = s1DualAscending.merge(s1DualDescending).select('VV').mean().reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=10).get('VV').getInfo()
        values = [vhIwDescMean,vvIwAscDescMean]
        #values = [vhIwAscMean,vhIwDescMean,vvIwAscDescMean]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')

    return monthly_mean_values

In order to prevent Google Colab from crashing due to memory limitations, I've initially prepared the first 100 rows of data. The resulting output file has been saved with "_0to100" added to its name to signify that it contains the truncated input size. In subsequent runs, I'll be working with the next 100 rows of data, ... .

In [ ]:
# for Sentinel-2
iran_data_ = iran_data.iloc[0:100] #[400:500] #[300:400] #[200:300] #[100:200] #[0:100]
iran_data_

In [ ]:
# for Sentinel-1
#iran_data_ = iran_data

In [ ]:
%%time
# Extract mean pixel values
mean_pixel_values = iran_data_.apply(extract_monthly_mean_pixel_values_iran, axis=1, result_type='expand')
# Create a list of column names for all columns
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
# Set the column names for the mean_pixel_values DataFrame
mean_pixel_values.columns = column_names
train_data_iran = pd.concat([iran_data_, mean_pixel_values], axis=1)

train_data_iran.head()

In [ ]:
train_data_iran.to_csv('train_data_iran_2019_2020_month_0to100.csv', index = False)

In [ ]:
train_data_iran.to_csv('train_data_iran_sentinel1.csv', index = False)

In [ ]:
# Sentinel-2
sudan_data_ = sudan_data.iloc[0:100] #[400:500] #[300:400] #[200:300] #[100:200] #[0:100]
sudan_data_

In [ ]:
# Sentinel-1
#sudan_data_ = sudan_data

In [ ]:
%%time
mean_pixel_values = sudan_data_.apply(extract_monthly_mean_pixel_values_sudan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
train_data_sudan = pd.concat([sudan_data_, mean_pixel_values], axis=1)
train_data_sudan.head()

In [ ]:
train_data_sudan.to_csv('train_data_sudan_2019_2020_month_0to100.csv', index = False)

In [ ]:
train_data_sudan.to_csv('train_data_sudan_sentinel1.csv', index = False)

In [ ]:
# Sentinel-2
afghanistan_data_ = afghanistan_data.iloc[300:400] #[400:500] #[300:400] #[200:300] #[100:200] #[0:100]
afghanistan_data_

In [ ]:
# Sentinel-1
#afghanistan_data_ = afghanistan_data

In [ ]:
%%time
mean_pixel_values = afghanistan_data_.apply(extract_monthly_mean_pixel_values_afghanistan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
train_data_afghanistan = pd.concat([afghanistan_data_, mean_pixel_values], axis=1)
train_data_afghanistan.head()

In [ ]:
train_data_afghanistan.to_csv('train_data_afghanistan_2021_month_0to100.csv', index = False)

In [ ]:
train_data_afghanistan.to_csv('train_data_afghanistan_2021_month_sentinel1.csv', index = False)

### Compute a single average value for the entire time

In [ ]:
%%time
mean_pixel_values = iran_data.apply(extract_mean_pixel_values_iran, axis=1, result_type='expand')
mean_pixel_values.columns = bands
train_data_iran = pd.concat([iran_data, mean_pixel_values], axis=1)
train_data_iran.head()

In [ ]:
%%time
mean_pixel_values = sudan_data.apply(extract_mean_pixel_values_sudan, axis=1, result_type='expand')
mean_pixel_values.columns = bands
train_data_sudan = pd.concat([sudan_data, mean_pixel_values], axis=1)
train_data_sudan.head()

In [ ]:
%%time
mean_pixel_values = afghanistan_data.apply(extract_mean_pixel_values_afghanistan, axis=1, result_type='expand')
mean_pixel_values.columns = bands
train_data_afghanistan = pd.concat([afghanistan_data, mean_pixel_values], axis=1)
train_data_afghanistan.head()

In [ ]:
#train_data_afghanistan.to_csv('train_data_afghanistan_april.csv', index = False)
#train_data_afghanistan.to_csv('train_data_afghanistan_sentinel1.csv', index = False)

In [ ]:
data = test.copy()

# Filter the data for Afghanistan
afghanistan_data = data[(data['Lat'] >= afghanistan_lat_range[0]) & (data['Lat'] <= afghanistan_lat_range[1]) &
                        (data['Lon'] >= afghanistan_lon_range[0]) & (data['Lon'] <= afghanistan_lon_range[1])]

# Filter the data for Iran
iran_data = data[(data['Lat'] >= iran_lat_range[0]) & (data['Lat'] <= iran_lat_range[1]) &
                 (data['Lon'] >= iran_lon_range[0]) & (data['Lon'] <= iran_lon_range[1])]

# Filter the data for Sudan
sudan_data = data[(data['Lat'] >= sudan_lat_range[0]) & (data['Lat'] <= sudan_lat_range[1]) &
                  (data['Lon'] >= sudan_lon_range[0]) & (data['Lon'] <= sudan_lon_range[1])]

In [ ]:
# Sentinel-2
iran_data_ = iran_data.iloc[400:500] #[400:500]  #[300:400]  #[200:300]  #[100:200]   #[0:100]
iran_data_

In [ ]:
# Sentinel-1
#iran_data_ = iran_data

In [ ]:
mean_pixel_values = iran_data_.apply(extract_monthly_mean_pixel_values_iran, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
test_data_iran = pd.concat([iran_data_, mean_pixel_values], axis=1)

In [ ]:
test_data_iran.to_csv('test_data_iran_2019_2020_month_400to500.csv', index = False)
#test_data_iran.to_csv('test_data_iran_sentinel1.csv', index = False)

In [ ]:
# Sentinel-2
sudan_data_ = sudan_data.iloc[400:500] #[400:500]  #[300:400]  #[200:300]  #[100:200]   #[0:100]
sudan_data_

In [ ]:
# Sentinel-1
#sudan_data_ = sudan_data

In [ ]:
mean_pixel_values = sudan_data_.apply(extract_monthly_mean_pixel_values_sudan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
test_data_sudan = pd.concat([sudan_data_, mean_pixel_values], axis=1)

In [ ]:
test_data_sudan.to_csv('test_data_sudan_2019_2020_month_200to300.csv', index = False)
#test_data_sudan.to_csv('test_data_sudan_sentinel1.csv', index = False)

In [ ]:
afghanistan_data_ = afghanistan_data.iloc[400:500] #[400:500]  #[300:400]  #[200:300]  #[100:200]   #[0:100]
afghanistan_data_

# Sentinel-1
#afghanistan_data_ = afghanistan_data

In [ ]:
mean_pixel_values = afghanistan_data_.apply(extract_monthly_mean_pixel_values_afghanistan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
test_data_afghanistan = pd.concat([afghanistan_data_, mean_pixel_values], axis=1)

In [ ]:
#test_data_afghanistan.to_csv('test_data_afghanistan_2021_month_400to500.csv', index = False)
#test_data_afghanistan.to_csv('test_data_afghanistan_2021_month_sentinel1.csv', index = False)

In [ ]:
mean_pixel_values = sudan_data.apply(extract_mean_pixel_values_sudan, axis=1, result_type='expand')
mean_pixel_values.columns = bands
test_data_sudan = pd.concat([sudan_data, mean_pixel_values], axis=1)

In [ ]:
mean_pixel_values = iran_data.apply(extract_mean_pixel_values_iran, axis=1, result_type='expand')
mean_pixel_values.columns = bands
test_data_iran = pd.concat([iran_data, mean_pixel_values], axis=1)

In [ ]:
mean_pixel_values = afghanistan_data.apply(extract_mean_pixel_values_afghanistan, axis=1, result_type='expand')
mean_pixel_values.columns = bands
test_data_afghanistan = pd.concat([afghanistan_data, mean_pixel_values], axis=1)

In [ ]:
#test_data_afghanistan.to_csv('test_data_afghanistan_sentinel1.csv', index = False)
#test_data_afghanistan.to_csv('test_data_afghanistan_april.csv', index = False)

# <font color="orange"> Landsat data preprocessing

In [ ]:
# Mask pixels based on various bitmask flags (CLOUD & SHADOW, CLEAR, ETC.)
def maskL8sr(image):
    # Landsat 8 T1 SR ‘pixel_qa’ band bitmasks:

    # Fill bitmask (bit 0), fill_quality
    fillBitMask = 1 << 0
    # Clear bitmask (bit 1), clear_quality
    clearBitMask = 1 << 1
    # Water bitmask (bit 2), water_quality
    waterBitMask = 1 << 2
    # Cloud shadow bitmask (bit 3), cloud_shadow_quality
    cloudShadowBitMask = 1 << 3    # Same as np.power(2, 3) or ee.Number(2).pow(3).getInfo() or 1000 in base 2
    # Snow bitmask (bit 4), snow_quality
    snowBitMask = 1 << 4
    # Cloud bitmask (bit 5), cloud_quality
    cloudsBitMask = 1 << 5         # Same as np.power(2, 5) or ee.Number(2).pow(5).getInfo() or 100000 in base 2
    # Cloud confidence bitmask (bits 6-7), cloud_confidence_quality
    cloudConfidenceBitMask = 1 << 6; cloudConfidenceBitMask_ = 1 << 7
    # Cirrus confidence bitmask (bits 8-9), cirrus_confidence_quality
    cirrusConfidenceBitMask = 1 << 8; cirrusConfidenceBitMask_ = 1 << 9
    # Terrain occlusion bitmask (bit 10), terrain_occlusion_quality
    terrainBitMask = 1 << 10

    # Select pixel QA band
    qa = image.select('pixel_qa')

    # shadow and cloud flags should be set to zero, indicating clear conditions (shadow free AND cloud free)
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

    # Return the masked image, scaled to reflectance, without the QA bands
    return image.updateMask(mask).divide(10000) \
          .select("B[0-9]*").copyProperties(image, ["system:time_start"])

In [ ]:
l8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')    # landsat 8
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

start_date_afghanistan = ee.Date('2022-04-01')
end_date_afghanistan = ee.Date('2022-04-30')

start_date_iran = ee.Date('2019-07-01')
end_date_iran = ee.Date('2020-06-30')

start_date_sudan = ee.Date('2019-07-01')
end_date_sudan = ee.Date('2020-06-30')

def extract_mean_pixel_values_afghanistan(row):
    lon = row['Lon']
    lat = row['Lat']

    values = None

    point = ee.Geometry.Point(lon, lat)

    la_filtered = l8_collection.filterBounds(point).filterDate(start_date_afghanistan, end_date_afghanistan).filterMetadata('CLOUD_COVER','less_than',20).map(maskL8sr)

    # Calculate the mean pixel values for the bands of interest at the labeled location
    mean_values = la_filtered.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
    )

    if la_filtered.size().getInfo() == 0:
       mean_values = None

    if mean_values is not None:
       values = [mean_values.get(band).getInfo() for band in bands]

    return values

In [ ]:
l8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

start_date_afghanistan = ee.Date('2021-04-01')
end_date_afghanistan = ee.Date('2022-03-31')

start_date_iran = ee.Date('2019-07-01')
end_date_iran = ee.Date('2020-06-30')

start_date_sudan = ee.Date('2019-07-01')
end_date_sudan = ee.Date('2020-06-30')

def extract_monthly_mean_pixel_values_afghanistan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_afghanistan

    while current_date.getInfo()['value'] <= end_date_afghanistan.getInfo()['value']:
        l8_filtered = l8_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month')).filterMetadata('CLOUD_COVER','less_than',20).map(maskL8sr)

        mean_values = l8_filtered.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=30
        )

        if l8_filtered.size().getInfo() == 0:
           mean_values = None

        if mean_values is not None:
          values = [mean_values.get(band).getInfo() for band in bands]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')
    return monthly_mean_values

def extract_monthly_mean_pixel_values_iran(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_iran

    while current_date.getInfo()['value'] <= end_date_iran.getInfo()['value']:
        l8_filtered = l8_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month')).filterMetadata('CLOUD_COVER','less_than',20).map(maskL8sr)

        mean_values = l8_filtered.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=30
        )

        if l8_filtered.size().getInfo() == 0:
           mean_values = None

        if mean_values is not None:
          values = [mean_values.get(band).getInfo() for band in bands]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')
    return monthly_mean_values

def extract_monthly_mean_pixel_values_sudan(row):
    lon = row['Lon']
    lat = row['Lat']

    point = ee.Geometry.Point(lon, lat)

    monthly_mean_values = []

    current_date = start_date_sudan

    while current_date.getInfo()['value'] <= end_date_sudan.getInfo()['value']:
        l8_filtered = l8_collection.filterBounds(point).filterDate(current_date, current_date.advance(1, 'month')).filterMetadata('CLOUD_COVER','less_than',20).map(maskL8sr)

        mean_values = l8_filtered.mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=30
        )

        if l8_filtered.size().getInfo() == 0:
           mean_values = None

        if mean_values is not None:
          values = [mean_values.get(band).getInfo() for band in bands]

        monthly_mean_values.extend(values)

        current_date = current_date.advance(1, 'month')
    return monthly_mean_values

In [ ]:
afghanistan_data_ = afghanistan_data.iloc[250:500] #[0:250]  #[250:500]

mean_pixel_values = afghanistan_data_.apply(extract_monthly_mean_pixel_values_afghanistan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
train_data_afghanistan = pd.concat([afghanistan_data_, mean_pixel_values], axis=1)

train_data_afghanistan.head()

In [ ]:
#train_data_afghanistan.to_csv('train_data_afghanistan_LANDSAT8_0to250.csv', index = False)

In [ ]:
'''
%%time
mean_pixel_values = afghanistan_data.apply(extract_mean_pixel_values_afghanistan, axis=1, result_type='expand')
mean_pixel_values.columns = bands
train_data_afghanistan = pd.concat([afghanistan_data, mean_pixel_values], axis=1)
train_data_afghanistan.head()

In [ ]:
iran_data_ = iran_data.iloc[100:500]   #[100:500]  #[0:100]

mean_pixel_values = iran_data_.apply(extract_monthly_mean_pixel_values_iran, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
train_data_iran = pd.concat([iran_data_, mean_pixel_values], axis=1)

train_data_iran.head()

In [ ]:
train_data_iran.to_csv('train_data_iran_LANDSAT8_100to500.csv', index = False)

In [ ]:
sudan_data_ = sudan_data.iloc[0:250]  #[250:500]  #[0:250]

mean_pixel_values = sudan_data_.apply(extract_monthly_mean_pixel_values_sudan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
train_data_sudan = pd.concat([sudan_data_, mean_pixel_values], axis=1)

train_data_sudan.head()

In [ ]:
train_data_sudan.to_csv('train_data_sudan_LANDSAT8_0to250.csv', index = False)

In [ ]:
data = test.copy()

# Filter the data for Afghanistan
afghanistan_data = data[(data['Lat'] >= afghanistan_lat_range[0]) & (data['Lat'] <= afghanistan_lat_range[1]) &
                        (data['Lon'] >= afghanistan_lon_range[0]) & (data['Lon'] <= afghanistan_lon_range[1])]

# Filter the data for Iran
iran_data = data[(data['Lat'] >= iran_lat_range[0]) & (data['Lat'] <= iran_lat_range[1]) &
                 (data['Lon'] >= iran_lon_range[0]) & (data['Lon'] <= iran_lon_range[1])]

# Filter the data for Sudan
sudan_data = data[(data['Lat'] >= sudan_lat_range[0]) & (data['Lat'] <= sudan_lat_range[1]) &
                  (data['Lon'] >= sudan_lon_range[0]) & (data['Lon'] <= sudan_lon_range[1])]

In [ ]:
mean_pixel_values = afghanistan_data.apply(extract_monthly_mean_pixel_values_afghanistan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
test_data_afghanistan = pd.concat([afghanistan_data, mean_pixel_values], axis=1)

In [ ]:
#test_data_afghanistan.to_csv('test_data_afghanistan_LANDSAT8.csv', index = False)

In [ ]:
'''
mean_pixel_values = afghanistan_data.apply(extract_mean_pixel_values_afghanistan, axis=1, result_type='expand')
mean_pixel_values.columns = bands
test_data_afghanistan = pd.concat([afghanistan_data, mean_pixel_values], axis=1)

In [ ]:
mean_pixel_values = iran_data.apply(extract_monthly_mean_pixel_values_iran, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
test_data_iran = pd.concat([iran_data, mean_pixel_values], axis=1)

In [ ]:
test_data_iran.to_csv('test_data_iran_LANDSAT8.csv', index = False)

In [ ]:
mean_pixel_values = sudan_data.apply(extract_monthly_mean_pixel_values_sudan, axis=1, result_type='expand')
column_names = [f'{band}_month{month}' for month in range(1, 13) for band in bands]
mean_pixel_values.columns = column_names
test_data_sudan = pd.concat([sudan_data, mean_pixel_values], axis=1)

In [ ]:
test_data_sudan.to_csv('test_data_sudan_LANDSAT8.csv', index = False)

*************************